In [1]:
import json
import random
train_path = "make_data.json" # from https://github.com/urchade/GLiNER/blob/main/examples/sample_data.json

with open(train_path, "r") as f:
    data = json.load(f)

print('Dataset size:', len(data))
#shuffle
random.shuffle(data)    
print('Dataset is shuffled...')

train_data = data[:int(len(data)*0.9)]
test_data = data[int(len(data)*0.9):]

print('Dataset is splitted...')

Dataset size: 25
Dataset is shuffled...
Dataset is splitted...


In [9]:
ners = list(set([j[-1] for i in train_data for j in i["ner"]]))

In [10]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
import torch
import sys

# 현재 작업 디렉토리의 경로를 sys.path에 추가
current_dir = os.getcwd()
gliner_dir = os.path.join(current_dir, "../")

# gliner 폴더를 sys.path에 추가하여 최우선 참조
sys.path.insert(0, gliner_dir)

from gliner import GLiNERConfig, GLiNER
from gliner.data_processing.collator import DataCollatorWithPadding
from gliner import GLiNER
from gliner.training import Trainer, TrainingArguments
from gliner.utils import load_config_as_namespace
from gliner.data_processing import WordsSplitter, GLiNERDataset
from gliner.data_processing import GLiNERDataset

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model_name = "taeminlee/gliner_ko"
cache_dir = "/home/dothis/dothis-ai/models/huggingface"
model = GLiNER.from_pretrained(model_name, cache_dir=cache_dir)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/home/dothis/anaconda3/envs/csu/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [11]:
train_dataset = GLiNERDataset(train_data, model.config, data_processor=model.data_processor)
test_dataset = GLiNERDataset(test_data, model.config, data_processor=model.data_processor)

data_collator = DataCollatorWithPadding(model.config)

100%|██████████| 22/22 [00:00<00:00, 165366.82it/s]


Total number of entity classes:  10


100%|██████████| 3/3 [00:00<00:00, 68385.39it/s]

Total number of entity classes:  4


In [12]:
#Optional: compile model for faster training
torch.set_float32_matmul_precision('high')
model.to(device)
model.compile_for_training()

Compiling transformer encoder...
Compiling RNN...
Compiling span representation layer...
Compiling prompt representation layer...


In [13]:
import torch._dynamo

# learning_rate는 학습 속도를 결정하는 중요한 하이퍼파라미터로, 모델이 학습 과정에서 얼마나 빠르게 가중치를 업데이트할지 결정합니다. 
# 현재 설정된 학습률은 5e-6이므로, 이를 조금 더 높은 값으로 변경해보세요. 예를 들어, 1e-5, 5e-5 또는 1e-4 등의 값을 시도할 수 있습니다. 
# 그러나 너무 큰 값으로 설정하면 학습이 불안정해질 수 있으므로 점진적으로 값을 증가시키는 것이 좋습니다.
# warmup_ratio: 현재는 0.1로 설정되어 있는데, 이 값을 조정하여 학습 초기 단계에서 학습률을 점진적으로 증가시키는 양을 조정할 수 있습니다. 
# 예를 들어, 0.2로 증가시키면 더 천천히 학습률이 최대치에 도달하게 됩니다.
# lr_scheduler_type: linear 외에도 cosine, cosine_with_restarts, polynomial 등 다양한 학습률 스케줄러를 시도해 볼 수 있습니다. 
# 각각의 스케줄러는 학습률을 조절하는 방식이 다르므로, 모델의 성능에 영향을 줄 수 있습니다.

training_args = TrainingArguments(
    output_dir="models",
    learning_rate=1e-6,
    weight_decay=0.01,
    others_lr=1e-5,
    others_weight_decay=0.01,
    lr_scheduler_type="linear", #cosine
    warmup_ratio=0.2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_steps = 1000,
    save_total_limit=10,
    dataloader_num_workers = 8,
    use_cpu = False,
    report_to="none",
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=model.data_processor.transformer_tokenizer,
    data_collator=data_collator,
)

torch._dynamo.config.suppress_errors = True
trainer.train()

/home/dothis/anaconda3/envs/csu/lib/python3.8/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
[2024-07-03 10:24:42,893] [3/0] torch._inductor.fx_pass

In [6]:
# 학습이 끝난 후 모델 저장
output_dir = "models"  # 저장할 디렉토리 경로 설정
trainer.save_model(output_dir)

In [7]:
output_dir = "models"  # 저장할 디렉토리 경로 설정
md = GLiNER.from_pretrained(output_dir, load_tokenizer=False)

config.json not found in /home/dothis/dothis-ai/code/preprocessing/GLiNER/examples/models
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
text = "새벽5시 PC방식당 라면 먹방 간짜장 열라면 탕수육 까르보나라 피자 돈까스 삼겹살 짜파게티 비빔면 김치볶음밥 로제파스타 고로케 소떡소떡 와플 ramen mukbang"
       

# Labels for entity prediction

tta_labels = ["tokens", "artifacts", "person", "animal", "CIVILIZATION", "organization", \
        "phone number", "address", "passport number", "email", "credit card number", \
        "social security number", "health insurance id number", 'Business/organization', \
        "mobile phone number", "bank account number", "medication", "cpf", "driver's license number", \
        "tax identification number", "medical condition", "identity card number", "national id number", \
        "ip address", "email address", "iban", "credit card expiration date", "username", \
        "health insurance number", "student id number", "insurance number", "activity/event", \
        "landline phone number", "blood type", "cvv", "Food", "Brand", "resource", "FACILITY", \
        "digital signature", "social media handle", "license plate number", "cnpj", "postal code", \
        "passport_number", "vehicle registration number", "credit card brand", "food", "Restaurant", \
        "fax number", "visa number", "insurance company", "identity document number", \
        "national health insurance number", "cvc", "birth certificate number", "train ticket number", \
        "passport expiration date", "social_security_number", "EVENT", "STUDY_FIELD", "LOCATION", \
        "MATERIAL", "PLANT", "TERM", "THEORY", 'Analysis Requirement', 'media content']
# labels = ["Person", "Award"] # for v2.1 use capital case for better performance

# Perform entity prediction
entities = md.predict_entities(text, tta_labels, threshold=0.0)
# entities = model.predict_entities(text, tta_labels, threshold=0.0)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"], "=>", entity["score"])
    print()

새벽5시 => address => 0.08877214044332504

PC방식당 => insurance company => 0.7874050736427307

라면 => CIVILIZATION => 0.9401974678039551

먹방 => credit card number => 0.05449414998292923

간짜장 => CIVILIZATION => 0.9794231057167053

열라면 => CIVILIZATION => 0.974287211894989

탕수육 => CIVILIZATION => 0.9740515351295471

까르보나라 => CIVILIZATION => 0.9535607099533081

피자 => CIVILIZATION => 0.938845157623291

돈까스 => CIVILIZATION => 0.9067108631134033

삼겹살 => CIVILIZATION => 0.9629043340682983

짜파게티 => CIVILIZATION => 0.9276237487792969

비빔면 => CIVILIZATION => 0.9615005850791931

김치볶음밥 => CIVILIZATION => 0.9626264572143555

로제파스타 => CIVILIZATION => 0.9597557783126831

고로케 => CIVILIZATION => 0.969813346862793

소떡소떡 => CIVILIZATION => 0.9767699837684631

와플 => CIVILIZATION => 0.9515304565429688

ramen => CIVILIZATION => 0.6979901194572449

mukbang => CIVILIZATION => 0.5251654386520386



In [16]:
model = GLiNER.from_pretrained(model_name, cache_dir=cache_dir)
entities = model.predict_entities(text, tta_labels, threshold=0.0)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"], "=>", entity["score"])
    print()

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


새벽5시 => address => 0.0657939463853836

PC방식당 => insurance company => 0.8107811212539673

라면 => CIVILIZATION => 0.9729574918746948

먹방 => credit card number => 0.004040951374918222

간짜장 => CIVILIZATION => 0.9905232787132263

열라면 => CIVILIZATION => 0.9863773584365845

탕수육 => CIVILIZATION => 0.9846587181091309

까르보나라 => CIVILIZATION => 0.9652066230773926

피자 => CIVILIZATION => 0.9571497440338135

돈까스 => CIVILIZATION => 0.9248909950256348

삼겹살 => CIVILIZATION => 0.9747151136398315

짜파게티 => CIVILIZATION => 0.9472574591636658

비빔면 => CIVILIZATION => 0.973558247089386

김치볶음밥 => CIVILIZATION => 0.9727054834365845

로제파스타 => CIVILIZATION => 0.9755313992500305

고로케 => CIVILIZATION => 0.9824250340461731

소떡소떡 => CIVILIZATION => 0.9859128594398499

와플 => CIVILIZATION => 0.9689881801605225

ramen => CIVILIZATION => 0.7187020778656006

mukbang => CIVILIZATION => 0.5153679251670837

